# Requirements Agent MVP Demo (Detailed) — Primary Loop Valves

This notebook is a **hands-on, end-to-end demo** of the current MVP for a *deterministic* requirements agent focused on **nuclear primary-loop valves**.

## 1) Setup


In [57]:
from pathlib import Path
from typing import Optional
import sys, zipfile

# Where is the notebook running from?
CWD = Path.cwd().resolve()

# If we're in repo_root/demo, start searching from repo_root
seed = CWD.parent if CWD.name.lower() == "demo" else CWD

def find_repo_root(start: Path) -> Path:
    """
    Walk upward looking for common repo markers OR our project markers.
    """
    markers = [".git", "pyproject.toml", "setup.cfg", "requirements.txt", "README.md"]
    for p in (start, *start.parents):
        if any((p / m).exists() for m in markers):
            return p
        # also accept either MVP layout as a "root"
        if (p / "requirements_agent_mvp" / "src" / "engine.py").exists():
            return p
        if (p / "src" / "engine.py").exists():
            return p
    return start  # fallback

REPO_ROOT = find_repo_root(seed)

def has_integrated_layout(root: Path) -> bool:
    return (root / "src" / "engine.py").exists()

def has_bundle_layout(root: Path) -> bool:
    return (root / "requirements_agent_mvp" / "src" / "engine.py").exists()

# Decide where the importable package lives
if has_integrated_layout(REPO_ROOT):
    BASE_DIR = REPO_ROOT
elif has_bundle_layout(REPO_ROOT):
    BASE_DIR = REPO_ROOT / "requirements_agent_mvp"
else:
    # Try to unzip a bundle from repo root (newest wins)
    bundles = sorted(
        REPO_ROOT.glob("requirements_agent_mvp_valves_v*.zip"),
        key=lambda p: p.stat().st_mtime,
        reverse=True
    )
    if bundles:
        zip_path = bundles[0]
        print(f"Unzipping bundle into repo root: {zip_path.name}")
        with zipfile.ZipFile(zip_path, "r") as z:
            z.extractall(REPO_ROOT)

    # Re-check after unzip
    if has_bundle_layout(REPO_ROOT):
        BASE_DIR = REPO_ROOT / "requirements_agent_mvp"
    elif has_integrated_layout(REPO_ROOT):
        BASE_DIR = REPO_ROOT
    else:
        raise FileNotFoundError(
            "Could not locate MVP code.\n"
            "Expected either:\n"
            f"  - {REPO_ROOT}/src/engine.py (integrated layout), or\n"
            f"  - {REPO_ROOT}/requirements_agent_mvp/src/engine.py (bundle layout), or\n"
            "  - a zip bundle named requirements_agent_mvp_valves_v*.zip in repo root.\n\n"
            f"Current CWD: {CWD}\n"
            f"Computed REPO_ROOT: {REPO_ROOT}\n"
            f"REPO_ROOT contents: {[p.name for p in list(REPO_ROOT.iterdir())[:30]]}"
        )

# Demo outputs go under repo_root/demo/out (or current demo folder if it exists)
DEMO_DIR = (REPO_ROOT / "demo") if (REPO_ROOT / "demo").exists() else CWD
OUT_DIR = DEMO_DIR / "out"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Add BASE_DIR to path for imports (src.engine, etc.)
if str(BASE_DIR) not in sys.path:
    sys.path.insert(0, str(BASE_DIR))

print("CWD     :", CWD)
print("REPO_ROOT:", REPO_ROOT)
print("BASE_DIR:", BASE_DIR)
print("OUT_DIR :", OUT_DIR)
print("sys.path[0]:", sys.path[0])


CWD     : /home/tanome/projects/zz_stash/requirements_agent/demo
REPO_ROOT: /home/tanome/projects/zz_stash/requirements_agent
BASE_DIR: /home/tanome/projects/zz_stash/requirements_agent
OUT_DIR : /home/tanome/projects/zz_stash/requirements_agent/demo/out
sys.path[0]: /home/tanome/projects/zz_stash/requirements_agent


## 2) Load inputs

We load two primary inputs:

1. **Requirements Library (template)**: `data/primary_loop_valve_baseline_v0.2_150reqs.json`
   - This contains *template requirements* (a superset).
   - Each requirement has applicability logic (`applicability.when`).

2. **ValveProfile (instance description)**: `examples/valve_profile_example.json`
   - This is the structured description of *one* valve.
   - Applicability logic is evaluated against this profile.

Optionally, we can validate the ValveProfile against the JSON Schema.


In [62]:

import json
from pathlib import Path

template_path = BASE_DIR / "data" / "primary_loop_valve_baseline_v0.2_150reqs.json"
profile_path  = BASE_DIR / "demo" / "valve_profile_example.json"

with open(template_path, "r", encoding="utf-8") as f:
    template = json.load(f)

with open(profile_path, "r", encoding="utf-8") as f:
    valve_profile = json.load(f)

print("Template ID :", template.get("template_id"))
print("Valve tag   :", valve_profile.get("valve_tag"))
print("Valve type  :", valve_profile.get("valve_type"))
print("Function    :", valve_profile.get("function"))
print("Actuation   :", valve_profile.get("actuation_type"))


Template ID : primary_loop_valve_baseline_v0.2_150reqs
Valve tag   : RCS-VLV-001
Valve type  : gate
Function    : isolation
Actuation   : MOV


### Optional: Schema validation

If `jsonschema` is installed in your environment, this cell validates the ValveProfile against:
- `schemas/valve_profile.schema.json`

If `jsonschema` is not installed, the cell will skip validation.


In [63]:

schema_path = BASE_DIR / "schemas" / "valve_profile.schema.json"
with open(schema_path, "r", encoding="utf-8") as f:
    valve_profile_schema = json.load(f)

try:
    import jsonschema
    jsonschema.validate(instance=valve_profile, schema=valve_profile_schema)
    print("✅ ValveProfile validated against schema.")
except ImportError:
    print("ℹ️ jsonschema not installed; skipping schema validation.")
except Exception as e:
    print("❌ ValveProfile schema validation failed:")
    print(e)


✅ ValveProfile validated against schema.


## 3) Understand the applicability language (MVP)

Each requirement uses `applicability.when` as a list of conditions.

**Semantics:**
- The list is **AND**: all conditions in the list must match.
- For equality comparisons, a `|` means **OR**.

**Supported forms (current MVP):**
- `always`
- `key=value1|value2|value3`
- numeric comparisons: `key>number`, `key>=number`, `key<number`, `key<=number`

Example:
```json
"when": ["safety_classification=safety_related", "actuation_type=MOV|AOV|SOV"]
```
This requirement applies only if the valve is safety-related AND actuated by MOV/AOV/SOV.


In [70]:
instance = filter_and_instantiate(template, valve_profile)

for k in instance.keys():
  print(k)

instance["applicable_requirements"]

instance_id
template_id
generated_utc
valve_profile
summary
applicable_requirements
non_applicable_requirements
validation


[{'id': 'PLV-001',
  'text': 'The project shall define and record the licensing basis and code-of-record for the valve, including applicable regulations, incorporated codes/standards, and edition/addenda where required.',
  'type': 'programmatic',
  'verification': {'method': ['inspection'],
   'acceptance': 'Documented licensing basis and code-of-record exists and is under configuration control.'},
  'provenance_refs': ['CFR_10_50_55a'],
  'status': 'draft',
  'parameter_values': {},
  'tbd_parameters': [],
  'applicability': {'conditions': ['always'], 'matched': True}},
 {'id': 'PLV-002',
  'text': 'The valve’s safety function(s), required position(s) for each analyzed event, and success criteria shall be explicitly defined.',
  'type': 'programmatic',
  'verification': {'method': ['inspection'],
   'acceptance': 'Safety function(s), positions, and success criteria are documented and traceable to the design basis.'},
  'provenance_refs': ['CFR_10_50_AppA'],
  'status': 'draft',
  'pa

## 4) Run the deterministic pipeline

Now we run the **core MVP**:

1. Iterate all requirements in the library
2. Evaluate applicability conditions against the ValveProfile
3. Append a `validation` block (quality gate)


In [71]:
from src.engine import filter_and_instantiate

instance = filter_and_instantiate(template, valve_profile)

# Quick glance
instance["summary"], instance["validation"]["overall_status"]


({'applicable_count': 131,
  'non_applicable_count': 19,
  'tbd_parameter_count': 0},
 'pass')

## 5) Explore outputs

The result is a **ValveRequirementsInstance** JSON with two main lists:

- `applicable_requirements`: requirements that apply, with instantiated text
- `non_applicable_requirements`: requirements that do not apply, with reasons

### Summary


In [72]:

instance["summary"]


{'applicable_count': 131, 'non_applicable_count': 19, 'tbd_parameter_count': 0}

### Count requirements by category (template-level)

This helps you understand how the library is structured and where requirements cluster.


In [73]:

# Count requirements per category in the template library
category_counts = []
for req_set in template.get("requirement_sets", []):
    category_counts.append((req_set.get("category"), len(req_set.get("requirements", []))))

category_counts


[('1. Programmatic, classification, and design basis', 15),
 ('2. Pressure boundary integrity and structural design', 18),
 ('3. Interfaces with piping systems and installation loads', 12),
 ('4. Materials, cleanliness, and compatibility', 15),
 ('5. Functional performance (flow, leakage, operability)', 20),
 ('6. Actuation, controls, and power (when applicable)', 15),
 ('7. Seismic and environmental qualification', 15),
 ('8. Examination, testing, and surveillance provisions (factory + inservice)',
  20),
 ('9. Quality assurance, procurement, records, and configuration control', 10),
 ('10. Maintainability, accessibility, and operational considerations', 10)]

### Peek at a few applicable requirements

Each applicable requirement includes:
- `id`: stable ID from the library
- `text`: instantiated requirement text
- `verification`: method + acceptance criteria
- `parameter_values`: values used for placeholder substitution
- `tbd_parameters`: unresolved placeholders requiring follow-up
- `status`: `draft` or `review_required`


In [50]:

# Show the first 5 applicable requirements (compact view)
for r in instance["applicable_requirements"][:5]:
    print(f"{r['id']} | status={r['status']} | TBDs={r['tbd_parameters']}")
    print("  ", r["text"])
    print("  verify:", r["verification"]["method"], "|", r["verification"]["acceptance"])
    print()


PLV-001 | status=draft | TBDs=[]
   The project shall define and record the licensing basis and code-of-record for the valve, including applicable regulations, incorporated codes/standards, and edition/addenda where required.
  verify: ['inspection'] | Documented licensing basis and code-of-record exists and is under configuration control.

PLV-002 | status=draft | TBDs=[]
   The valve’s safety function(s), required position(s) for each analyzed event, and success criteria shall be explicitly defined.
  verify: ['inspection'] | Safety function(s), positions, and success criteria are documented and traceable to the design basis.

PLV-003 | status=draft | TBDs=[]
   The valve shall be classified (code class, safety classification, seismic category, QA level) using the project’s approved classification methodology.
  verify: ['inspection'] | Classification is documented, reviewed, approved, and traceable to project methodology.

PLV-004 | status=draft | TBDs=[]
   Design inputs for normal

### Why is a requirement not applicable?

For each non-applicable requirement, the engine stores:
- `conditions`: the `when` conditions
- `reasons`: human-readable reasons showing what did not match


In [51]:

# Show one non-applicable requirement and the reason(s)
instance["non_applicable_requirements"][0]


{'id': 'PLV-036',
 'conditions': ['valve_type=check'],
 'reasons': ["valve_type='gate' not in ['check']"]}

## 6) Quality gate (validation)

The quality gate runs automatically and produces `instance['validation']`.

### What is checked (current MVP)
- `verification.method` is present and non-empty
- `verification.acceptance` is present and non-empty
- unresolved placeholders are tracked in `tbd_parameters`
- basic atomicity heuristics (warnings)

### How to interpret results
- **Errors**: should be treated as hard failures (bad output)
- **Warnings**: flags for human review (e.g., potential compound requirement)
- **Info**: reserved for optional messages (not used heavily yet)


In [52]:

validation = instance["validation"]
validation


{'overall_status': 'pass',
 'error_count': 0,
 'warning_count': 22,
 'info_count': 0,
 'issue_count': 22,
 'issues': [{'severity': 'warning',
   'code': 'REQ_ATOMICITY_MULTI_SHALL',
   'message': "Requirement contains 2 occurrences of 'shall'; may be compound.",
   'requirement_id': 'PLV-014'},
  {'severity': 'warning',
   'code': 'REQ_ATOMICITY_CONJUNCTION',
   'message': "Requirement may be compound (contains 'shall ... and ... shall ...'); consider splitting.",
   'requirement_id': 'PLV-014'},
  {'severity': 'warning',
   'code': 'REQ_ATOMICITY_MULTI_SHALL',
   'message': "Requirement contains 2 occurrences of 'shall'; may be compound.",
   'requirement_id': 'PLV-026'},
  {'severity': 'warning',
   'code': 'REQ_ATOMICITY_CONJUNCTION',
   'message': "Requirement may be compound (contains 'shall ... and ... shall ...'); consider splitting.",
   'requirement_id': 'PLV-026'},
  {'severity': 'warning',
   'code': 'REQ_ATOMICITY_ANDOR',
   'message': "Requirement contains 'and/or' which i

### Aggregate validation issues by code

This is often more useful than raw issue lists, especially in CI.


In [53]:

from src.reporting import build_report

report = build_report(instance)
report["counts"], report["by_code"]


({'error_count': 0, 'warning_count': 22, 'info_count': 0, 'issue_count': 22},
 [{'severity': 'warning',
   'code': 'REQ_ATOMICITY_ANDOR',
   'count': 6,
   'requirement_ids': ['PLV-028',
    'PLV-043',
    'PLV-074',
    'PLV-082',
    'PLV-088',
    'PLV-112'],
   'message_examples': ["Requirement contains 'and/or' which is often ambiguous; consider splitting.",
    "Requirement contains 'and/or' which is often ambiguous; consider splitting.",
    "Requirement contains 'and/or' which is often ambiguous; consider splitting."]},
  {'severity': 'warning',
   'code': 'REQ_ATOMICITY_CONJUNCTION',
   'count': 8,
   'requirement_ids': ['PLV-014',
    'PLV-026',
    'PLV-031',
    'PLV-052',
    'PLV-056',
    'PLV-106',
    'PLV-109',
    'PLV-122'],
   'message_examples': ["Requirement may be compound (contains 'shall ... and ... shall ...'); consider splitting.",
    "Requirement may be compound (contains 'shall ... and ... shall ...'); consider splitting.",
    "Requirement may be compoun

## 7) Write demo outputs to `demo/out/`

We write three artifacts:
- `requirements_instance.json`: the full filtered/instantiated requirements instance
- `validation_report.json`: aggregated validation summary
- `validation_report.csv`: same summary in a spreadsheet-friendly format


In [54]:

from src.reporting import write_report_json, write_report_csv

instance_out = OUT_DIR / "requirements_instance.json"
report_json  = OUT_DIR / "validation_report.json"
report_csv   = OUT_DIR / "validation_report.csv"

with open(instance_out, "w", encoding="utf-8") as f:
    json.dump(instance, f, indent=2, ensure_ascii=False)

write_report_json(report, str(report_json))
write_report_csv(report, str(report_csv))

print("Wrote:", instance_out)
print("Wrote:", report_json)
print("Wrote:", report_csv)


Wrote: /home/tanome/projects/zz_stash/requirements_agent/demo/out/requirements_instance.json
Wrote: /home/tanome/projects/zz_stash/requirements_agent/demo/out/validation_report.json
Wrote: /home/tanome/projects/zz_stash/requirements_agent/demo/out/validation_report.csv


## 8) Strict mode demo (CLI)

The **CLI** is designed for automated pipelines.

Flags:
- `--strict` → return non-zero **only if errors exist**
- `--fail-on-warnings` → return non-zero if **any warnings exist**
- `--max-warnings N` → return non-zero if warnings exceed N

In the current library, many warnings are expected (atomicity heuristics). In CI you may choose:
- `--strict` for early stages, and
- later tighten to `--max-warnings` once you clean up atomicity.


In [55]:

import subprocess

cmd_base = [
    sys.executable, "-m", "src.engine",
    "--template", str(template_path),
    "--profile", str(profile_path),
    "--out", str(OUT_DIR / "instance_from_cli.json"),
    "--report-json", str(OUT_DIR / "report_from_cli.json"),
    "--report-csv", str(OUT_DIR / "report_from_cli.csv"),
]

# (A) strict: should pass unless there are validation errors
p_strict = subprocess.run(cmd_base + ["--strict"], cwd=str(BASE_DIR), capture_output=True, text=True)
print("Exit code (strict):", p_strict.returncode)
print("\n".join(p_strict.stdout.splitlines()[-6:]))

# (B) fail-on-warnings: likely fails today due to atomicity warnings
p_warn = subprocess.run(cmd_base + ["--fail-on-warnings"], cwd=str(BASE_DIR), capture_output=True, text=True)
print("\nExit code (fail-on-warnings):", p_warn.returncode)
print("\n".join(p_warn.stdout.splitlines()[-6:]))


Exit code (strict): 0
Wrote: /home/tanome/projects/zz_stash/requirements_agent/demo/out/instance_from_cli.json (applicable=131)
Wrote report (JSON): /home/tanome/projects/zz_stash/requirements_agent/demo/out/report_from_cli.json
Wrote report (CSV): /home/tanome/projects/zz_stash/requirements_agent/demo/out/report_from_cli.csv

Exit code (fail-on-warnings): 3
Wrote: /home/tanome/projects/zz_stash/requirements_agent/demo/out/instance_from_cli.json (applicable=131)
Wrote report (JSON): /home/tanome/projects/zz_stash/requirements_agent/demo/out/report_from_cli.json
Wrote report (CSV): /home/tanome/projects/zz_stash/requirements_agent/demo/out/report_from_cli.csv


## 9) Demo: missing parameters → TBD placeholders

To show how TBD tracking works, we intentionally remove some profile values.

**What to look for:**
- Requirements with placeholders keep the placeholder text (e.g., `{{dp_max}}`)
- Those placeholders are listed in `tbd_parameters`
- Requirement `status` becomes `review_required`
- Validation will warn that TBDs exist (but should not be an error if tracked correctly)


In [56]:

# Create a modified profile with missing parameters
profile_missing = dict(valve_profile)
profile_missing["dp_max"] = None
profile_missing["stroke_time_limit"] = None

instance_missing = filter_and_instantiate(template, profile_missing)

print("Summary:", instance_missing["summary"])
print("Validation overall:", instance_missing["validation"]["overall_status"])
print("Warning count:", instance_missing["validation"]["warning_count"])

# Find some requirements that now have TBD parameters
tbd_reqs = [r for r in instance_missing["applicable_requirements"] if r["tbd_parameters"]]
print("Num applicable requirements with TBDs:", len(tbd_reqs))

# Show one example
ex = tbd_reqs[0]
{
    "id": ex["id"],
    "status": ex["status"],
    "tbd_parameters": ex["tbd_parameters"],
    "text_preview": ex["text"][:250] + ("..." if len(ex["text"]) > 250 else "")
}


Summary: {'applicable_count': 131, 'non_applicable_count': 19, 'tbd_parameter_count': 2}
Validation overall: pass
Warning count: 25
Num applicable requirements with TBDs: 3


{'id': 'PLV-029',
 'status': 'review_required',
 'tbd_parameters': ['dp_max'],
 'text_preview': 'The valve shall withstand the maximum specified differential pressure {{dp_max}} without structural damage or permanent deformation that impairs function.'}